In [ ]:
import json

import cufflinks as cf
import numpy as np
import pandas as pd
from pandas.io.pytables import HDFStore

cf.go_offline()

In [ ]:
store = HDFStore("../store.h5")

data = []
for k in list(store.keys()):
    try:
        d = store.get(k)
        d["timestamp"] = pd.to_datetime(k.split("_")[1], format="%d%m%Y")
        data.append(d)
    except Exception as e:
        print("Problem reading", k)
        print(e)
store.close()

data = pd.concat(data)
data = data.set_index(["timestamp", "owner"])


In [ ]:
data_to_plot = data["size"].unstack().fillna(0)
dataplot = data_to_plot.iplot(kind="scatter", fill=True, asFigure=True)
data.iplot(data=dataplot["data"])

In [ ]:
latest_data = data[
    data.index.get_level_values("timestamp") == data.index.get_level_values("timestamp").max()
]
dataplot = latest_data.reset_index().iplot(
    kind="pie",
    labels="owner",
    values="size",
    hole=0.4,
    sort=True,
    textinfo="percent",
    asFigure=True,
)
dataplot["data"][0]["text"] = ["%.2f Gb" % xx for xx in dataplot["data"][0]["values"]]
dataplot["data"][0]["hoverinfo"] = "text+label"
data.iplot(data=dataplot["data"])